# Elastic Inference

Left the code here but it can not support pytorch 1.5.1 yet. If you want to test in the SageMaker Host Notebook. You need to mannaully install a conda env for pytorch_ei:

In termininal, run bash

```
conda create --name torchei_p36 --clone python3
conda activate torchei_p36
cd SageMaker/
wget "https://amazonei-pytorch.s3.amazonaws.com/torch_eia-1.3.1-cp36-cp36m-manylinux1_x86_64.whl"
pip install torch_eia-1.3.1-cp36-cp36m-manylinux1_x86_64.whl
pip install graphviz
```

Elastic inference change the torch.jit.optimized_execution interface. You will get the following error message in SageMaker Notebook:

```
optimized_execution() takes 1 positional argument but 2 were given
```

Here is discussion thread in stock overflow [link](https://stackoverflow.com/questions/60981262/optimized-execution-takes-1-positional-argument-but-2-were-given)

In [2]:
# load torchscript model
!cp ../2-yolov5-onnx/yolov5s.torchscript.pt .

In [1]:
import torch

img = torch.zeros((1, 3, 640, 640))

model = torch.jit.load('yolov5s.torchscript.pt', map_location=torch.device('cpu'))

# Perform inference. Make sure to disable autograd and use EI execution context
with torch.no_grad():
  with torch.jit.optimized_execution(True, {'target_device':'eia:0'}):
    probs = model(im)
    print(probs)

TypeError: optimized_execution() takes 1 positional argument but 2 were given

In [1]:
!/opt/amazon/ei/ei_tools/bin/ei describe-accelerators --json

{
  "ei_client_version": "1.6.3",
  "time": "Tue Jul 28 09:11:04 2020",
  "attached_accelerators": 1,
  "devices": [
    {
      "ordinal": 0,
      "type": "eia2.medium",
      "id": "eia-2b25d3cc118d4b79bac9b0668f27b0a6",
      "status": "healthy"
    }
  ]
}
